# 1. Создаем подключение к БД
Прямой ссылкой на файл, используя sqlalchemy (обязательный запуск)

In [1]:
import gdown
url = 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1E1QoF7Ey2anKN5UYusbYipnjao75Fo_Z'
output = '/content/data.db'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1E1QoF7Ey2anKN5UYusbYipnjao75Fo_Z
To: /content/data.db
100%|██████████| 16.0M/16.0M [00:00<00:00, 95.1MB/s]


'/content/data.db'

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine('sqlite:///data.db', connect_args={'check_same_thread': False})

# **Задание 1**
Необходимо с помощью SQL запроса собрать одну общую таблицу из всех данных витрин, которая должна включать в себя все поля этих витрин. В качестве ответа предоставить ссылку на вашу копию страницы, где можно увидеть написанный запрос и собранную таблицу. Ссылку вставьте в Word файл "Техническое задание" в соответствующее поле.

(одна из таблиц называется group, её имя конфликтует с зарезервированными ключевым словом SQL, поэтому его нужно будет заключить в двойные кавычки "")

In [7]:
query = text("""
WITH sales AS (
SELECT
	*
FROM
	call c
	JOIN task t ON c.wo_task_rk = t.task_rk
	JOIN action a ON c.wo_hit_rk = a.hit_rk
	JOIN product p ON c.wo_hit_rk = p.hit_rk
	JOIN emp_x_org_gr e ON c.wo_employee_rk = e.employee_rk
	JOIN queue q ON c.wo_queue_id = q.queue_id
	JOIN horoscope h ON c.agent_login = h.agent_login
),

management AS (
SELECT
	*
FROM
	mngmnt m
	JOIN "group" g ON m.org_management_rk = g.org_management_rk
),

sm AS (
SELECT
	*
FROM
	sales
	JOIN management ON sales.org_group_rk = management.org_group_rk
),

res AS (
SELECT
	*
FROM
	sm
	JOIN result ON sm.hit_status_result_id = result.hit_status_result_id
)

SELECT * FROM res;
""")
with engine.connect() as con:
  result = con.execute(query)
df = pd.DataFrame(result.fetchall())
df.to_csv('sales_db.csv', index=False)

# Задание 2
Необходимо посчитать следующие значения:


*   Общее количество звонков со статусом «Дозвон, Успешно»;
*   Продажная конверсия, по формуле:
Конверсия=NДозвон,Успешно / NДозвон,Успешно+NДозвон,Отказ
*   Средняя длительность одной коммуникации, закрытой статусом «Дозвон, Успешно»


Ответы необходимо вписать в соответствующую таблицу в фаиле "Техническое задание"

In [3]:
query = text("""
SELECT
	COUNT(action.hit_status_result_id) AS "Кол-во успешных дозвонов"
FROM
	action
	JOIN result ON action.hit_status_result_id = result.hit_status_result_id
WHERE
	result.hit_status_result_desc = 'Дозвон, Успешно';
  """
)
with engine.connect() as con:
  result = con.execute(query)
  count_suc_calls = result.scalar()
print(f'Количество успешных дозвонов: {count_suc_calls}')

Количество успешных дозвонов: 3310


In [5]:
query = text("""
WITH successful_calls AS (
	SELECT
		COUNT(action.hit_status_result_id) AS s_count
	FROM
		action
		JOIN result ON action.hit_status_result_id = result.hit_status_result_id
	WHERE
		result.hit_status_result_desc = 'Дозвон, Успешно'),

refusals AS (
	SELECT
		COUNT(action.hit_status_result_id) AS r_count
	FROM
		action
		JOIN result ON action.hit_status_result_id = result.hit_status_result_id
	WHERE
		result.hit_status_result_desc = 'Дозвон, Отказ')

SELECT
	(ROUND(CAST(s_count AS FLOAT) / (s_count + r_count)* 100, 2)) || '%' AS "Конверсия"
FROM
	successful_calls, refusals
"""
)
with engine.connect() as con:
  result = con.execute(query)
  conv = result.scalar()
print(f'Конверсия: {conv}')

Конверсия: 15.21%


In [6]:
query = text("""
SELECT
	FLOOR(AVG(duratoin_sec) / 60) AS "Минуты",
	ROUND(AVG(duratoin_sec) % 60, 0) AS "Секунды"
FROM
	call c
	JOIN action a ON a.hit_rk = c.wo_hit_rk
WHERE
	a.hit_status_result_id = (
		SELECT
			DISTINCT action.hit_status_result_id
		FROM
			action
			JOIN result ON action.hit_status_result_id = result.hit_status_result_id
		WHERE
			result.hit_status_result_desc = 'Дозвон, Успешно')
      """)
with engine.connect() as con:
  result = con.execute(query)
  row = result.first()
  length_min = row[0]
  length_sec = row[1]
formatted_length = f"{int(length_min)} минут {int(length_sec)} секунд"
print(f'Средняя продолжительность успешного звонка: {formatted_length}')

Средняя продолжительность успешного звонка: 1 минут 49 секунд
